In [5]:
import pandas as pd

In [6]:
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv('pong.csv')

In [8]:
df

,ball_x,ball_y,ball_vx,ball_vy,paddle_y
0,103,23,3,3,30
1,106,26,3,3,30
2,109,29,3,3,30
3,112,32,3,3,30
4,115,35,3,3,30
...,...,...,...,...,...
16416,778,147,3,-3,10
16417,781,144,3,-3,10
16418,784,141,3,-3,10
16419,787,138,3,-3,10
